# Principal Component Analysis

### Principal Component Analysis from Scratch

In [18]:
def parse_arff(file_path):
    data_started = False
    attributes = []
    data = []
    nominal_mapping = []
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()

            if not line or line.startswith('%'):
                continue

            if line.lower().startswith('@relation'):
                continue

            if line.lower().startswith('@attribute'):
                parts = line.split()
                attr_name = parts[1].strip()

                if '{' in line:
                    values = line[line.index('{') + 1:line.index('}')].split(',')
                    attributes.append((attr_name, 'nominal', values))
                    attribute_info = ((attr_name, 'nominal', values))
                    attr_name, attr_type, attr_values = attribute_info
                    nominal_mapping.append({value: index for index, value in enumerate(attr_values)})

                else:
                    attributes.append((attr_name, 'numeric', 0))

            if line.lower().startswith('@data'):
                data_started = True
                continue

            if data_started:
                data.append(line.split(','))

    return attributes, data

def standardize(matrix):
    means = [mean(col) for col in zip(*matrix)]
    std_devs = [((sum((x - means[i]) ** 2 for x in col) / len(col)) ** 0.5) for i, col in enumerate(zip(*matrix))]
    return [[(col[i] - means[i]) / std_devs[i] for i in range(len(col))] for col in matrix]

def mean(vector):
    return sum(vector) / len(vector)

def subtract_mean(matrix):
    col_means = [mean(col) for col in zip(*matrix)]
    return [[col - col_means[i] for i, col in enumerate(row)] for row in matrix]

def covariance_matrix(matrix):
    n = len(matrix)
    num_features = len(matrix[0])
    cov_matrix = [[0] * num_features for _ in range(num_features)]

    for i in range(num_features):
        for j in range(num_features):
            mean_i = mean(matrix[i])
            mean_j = mean(matrix[j])
            values_i = [float(val) for val in matrix[i]]
            values_j = [float(val) for val in matrix[j]]
            cov_matrix[i][j] = sum((val_i - mean_i) * (val_j - mean_j) for val_i, val_j in zip(values_i, values_j)) / (n - 1)

    return cov_matrix

def normalize(vector):
    norm = sum(x**2 for x in vector)**0.5
    return [x / norm for x in vector]

def multiply(matrix, vector):
    return [sum(x*y for x, y in zip(row, vector)) for row in matrix]

def eigenvalues_and_eigenvectors(matrix, num_simulations=1000):
    n = len(matrix)
    vec = [1] * n

    for _ in range(num_simulations):
        new_vec = multiply(matrix, vec)
        vec = normalize(new_vec)

    eigenvalue = sum(x*y for x, y in zip(multiply(matrix, vec), vec))
    eigenvector = vec

    return eigenvalue, eigenvector

def transform(matrix, eigenvectors, k):
    return [
        [sum(row[j] * eigenvectors[i][j] for j in range(len(row))) for i in range(k)]
        for row in matrix
    ]
    
def pca(matrix, k):
    num_features = len(matrix[0])
    standardized_matrix= standardize(matrix)
    cov_matrix = covariance_matrix(standardized_matrix)
    eigenvalues, eigenvectors = eigenvalues_and_eigenvectors(cov_matrix)
    sorted_indices = sorted(range(num_features), key=lambda k: eigenvalues, reverse=True)
    eigenvectors = [[eigenvectors[i] for j in sorted_indices] for i in range(num_features)]
    transformed_matrix = transform(standardized_matrix, eigenvectors, k)
    return transformed_matrix 

if __name__ == "__main__":
    file_paths = [
        r'C:\Users\Rieze\Desktop\2017.arff',
        r'C:\Users\Rieze\Desktop\2018.arff',
        r'C:\Users\Rieze\Desktop\2019.arff',
        r'C:\Users\Rieze\Desktop\2020.arff',
        r'C:\Users\Rieze\Desktop\2021 Q1.arff'
    ]
    
    data = []
    for file in file_paths:
        attributes, dt = parse_arff(file)
        for row in dt :
            for i in range ( len (attributes) ):
                attr_name, attr_type, attr_values = attributes[i]
                if attr_type == 'nominal' :
                    nominal_mapping = {value: index for index, value in enumerate(attr_values)}
                    row[i] = nominal_mapping.get(row[i])
                elif attr_type == 'numeric' :
                    try :
                        row[i] = float(row[i])
                    except Exception as e:
                        row[i] = 0
        for d in dt:
            data.append(d)


    num_components = 2
    transformed_data = pca(data, num_components)

    # print("Original Data:")
    # for sample in data:
    #    print(sample)
    print(f"\nTransformed Data Size: {len(transformed_data)}")
    print("\nTransformed Data (after PCA):")
    for sample in transformed_data:
        print(sample)


Transformed Data Size: 2250

Transformed Data (after PCA):
[-0.7414921542673345, -0.8526863111619523]
[-0.6769267096480548, -0.7784386330387173]
[-0.6988169582424298, -0.8036115431186611]
[-0.6248445003952293, -0.7185461761470625]
[-0.6906612149652025, -0.794232764651728]
[-0.6862387008012261, -0.7891470503028435]
[-0.6357893875055364, -0.7311323584316113]
[-0.5599842164213525, -0.6439594445622236]
[-0.6462439896980298, -0.7431547326764085]
[-0.36118411009578966, -0.4153472760506843]
[-0.7032949198380964, -0.8087610197956018]
[-0.6345320547505713, -0.7296864760678342]
[-0.6563016818450773, -0.7547206762488567]
[-0.6539203280644745, -0.7519822146763804]
[-0.6136731661392041, -0.7056995887048262]
[-0.6228341013476099, -0.7162342976760461]
[-0.6185507516837856, -0.7113086169345256]
[-0.6025648609899771, -0.6929254822136859]
[-0.5536092486363734, -0.6366284866646024]
[-0.6127421553917677, -0.7046289635939399]
[-0.4518278142656138, -0.5195839092959726]
[-0.6038010852752009, -0.694347090681

### SKLearn Principal Component Analysis

In [36]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np

# Assuming that 'X' is the data
X = np.array(data)

# Standardizing the features
X = StandardScaler().fit_transform(X)

pca = PCA(n_components=2)
principalComponents = pca.fit_transform(X)

print("Principal Components:")
print(principalComponents)


Principal Components:
[[-3.76228879 -0.22779347]
 [-3.87621957 -0.23015437]
 [-3.69906609 -0.22003422]
 ...
 [-3.72581704 -0.16837533]
 [-0.79783203 -0.11916585]
 [-3.76068789 -0.16835429]]


## Conclusion
The main cause of difference between my results and the results of sklearn is most likely the eigenvalue and eigenvector computation stage. The power iteration method I'm using may not be as precise or efficient as sklearn's methods. Furthermore, sklearn's PCA implementation contains a few extra steps, such as testing for n_components n_features and whitening (dividing the projected data by the square root of eigenvalues), which could result in different findings.

#### To be more specific
```
def eigenvalues_and_eigenvectors(matrix, num_simulations=1000):
    n = len(matrix)
    vec = [1] * n

    for _ in range(num_simulations):
        new_vec = multiply(matrix, vec)
        vec = normalize(new_vec)

    eigenvalue = sum(x*y for x, y in zip(multiply(matrix, vec), vec))
    eigenvector = vec

    return eigenvalue, eigenvector
```
I am using a power iteration method to calculate the eigenvalues and eigenvectors. This method, while generally effective, might not be as accurate or efficient as the methods used by sklearn. Sklearn uses LAPACK routines, which are highly optimized for these operations.

# Singular Value Decomposition

### SKLearn Singular Value Decomposition

In [41]:
import numpy as np
from sklearn.decomposition import TruncatedSVD

matrix = np.array(data)
n = len(matrix[0])

# Perform SVD
svd = TruncatedSVD(n_components=n-1)  # n_components should be less than the matrix size
svd.fit(matrix)

# Extract the components of SVD
U = svd.transform(matrix)
Sigma = np.diag(svd.singular_values_)
VT = svd.components_

# Reconstruct the matrix from the decomposed components
reconstructed_matrix = U @ Sigma @ VT

print("\nU matrix:")
print(U)

print("\nSigma matrix:")
print(Sigma)

print("\nVT matrix:")
print(VT)



U matrix:
[[ 6.64277182e-01  3.27834033e+05 -4.65978097e+00 ...  6.96577879e-03
   6.29809434e-01  2.56107593e-02]
 [ 3.71904578e+02  1.38334649e+05 -1.37249374e+00 ...  3.40033158e-01
   6.63016894e-01  9.34604844e-01]
 [ 3.93208236e+02  5.04710323e+03  9.26218353e-01 ...  5.88117908e-01
   1.63818688e-01  4.63452145e-01]
 ...
 [ 1.88986879e-07  1.68659983e-02  5.03933042e+00 ... -6.97167684e-01
   3.89286587e+00 -3.99710267e-01]
 [ 5.83618591e+01  1.04835794e+00  5.04017218e+00 ... -1.57948479e-01
   2.26143927e+00 -4.41256027e-01]
 [ 1.86083371e-07  1.70517944e-02  4.02008345e-02 ... -7.37591650e-01
   3.87623237e+00 -4.08882834e-01]]

Sigma matrix:
[[2.73380969e+09 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 6.83017599e+06 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 4.90233858e+06 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00

### Singular Value Decomposition from Scratch

In [ ]:
def transpose(matrix):
    return [[matrix[j][i] for j in range(len(matrix))] for i in range(len(matrix[0]))] if matrix else []

def svd(matrix):
    print("\n SVD is running... It might take a while")
    def multiply(matrix_a, matrix_b):
        return [[sum(x * y for x, y in zip(row_a, col_b)) for col_b in zip(*matrix_b)] for row_a in matrix_a]

    def gram_schmidt(matrix):
        def projection(u, v):
            return [x * sum(x_i * y_i for x_i, y_i in zip(u, v)) / sum(x_i**2 for x_i in u) for x in u]

        def subtract(v, projection):
            return [x - y for x, y in zip(v, projection)]

        basis = []
        for vector in matrix:
            for b in basis:
                vector = subtract(vector, projection(b, vector))
            basis.append(vector)
        return basis

    def normalize(vector):
        norm = sum(x ** 2 for x in vector) ** 0.5
        return [x / norm for x in vector]

    transposed_matrix = transpose(matrix)
    u_matrix = gram_schmidt(transposed_matrix)
    v_matrix = [normalize(multiply(matrix, u)) for u in u_matrix]

    sigma_matrix = multiply(transpose(u_matrix), matrix)
    sigma_matrix = multiply(sigma_matrix, transpose(v_matrix))

    return u_matrix, sigma_matrix, transpose(v_matrix)

u, sigma, v_t = svd(data)


 SVD is running... It might take a while


## Conclusion
The differences between my implementation and sklearn are data centering, dimensionality reduction, sparse matrices efficiency, and the algorithm used. Following sklearn, my approach does not involve any data centering. My version, on the other hand, computes the complete SVD, whereas sklearn SVD computes a reduced-rank approximation to the original matrix. Due to the use of operations such as matrix multiplication and transposition, sklearn is meant to be efficient with sparse matrices, but my implementation may not be as efficient with sparse matrices. Finally, for orthogonalization, my implementation use the Gram-Schmidt process, whereas sklearn employs either a fast randomized SVD solver or ARPACK.

```
def gram_schmidt(matrix):
        def projection(u, v):
            return [x * sum(x_i * y_i for x_i, y_i in zip(u, v)) / sum(x_i**2 for x_i in u) for x in u]

        def subtract(v, projection):
            return [x - y for x, y in zip(v, projection)]

        basis = []
        for vector in matrix:
            for b in basis:
                vector = subtract(vector, projection(b, vector))
            basis.append(vector)
        return basis
```

    